In [ ]:
'''We won't consider decision trump and simple decision tree due to lower results than other models by metric kaggle(accuracy): accordingly ~59% и ~70% unlike ~76%.'''

In [ ]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from functions_for_titanik import make_prediction_file

In [ ]:
#Path to data

PATH_TO_X_SAMPLE = "X_sample.csv"
PATH_TO_Y_SAMPLE = "y_sample.csv"

In [ ]:
#Loading data

X_sample = pd.read_csv(PATH_TO_X_SAMPLE, index_col="PassengerId")
y_sample = pd.read_csv(PATH_TO_Y_SAMPLE, index_col="PassengerId")